In [2]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

In [1]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/modelv4'

In [3]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
word_state_size = 100
other_state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 10

In [4]:
with tf.name_scope("input"):
    path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length")
    word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size], name="y")

In [5]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [6]:
with tf.name_scope("dropout"):
    h_drop = tf.nn.dropout(embedded_word, 0.3)

In [7]:
word_hidden_state = tf.zeros([batch_size, word_state_size], name='word_hidden_state')
word_cell_state = tf.zeros([batch_size, word_state_size], name='word_cell_state')
word_init_state = tf.contrib.rnn.LSTMStateTuple(word_hidden_state, word_cell_state)

other_hidden_states = tf.zeros([channels-1, batch_size, other_state_size], name="hidden_state")
other_cell_states = tf.zeros([channels-1, batch_size, other_state_size], name="cell_state")

other_init_states = [tf.contrib.rnn.LSTMStateTuple(other_hidden_states[i], other_cell_states[i]) for i in range(channels-1)]

with tf.variable_scope("word_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=word_init_state)
    state_series_word1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("word_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[1], sequence_length=path_length[1], initial_state=word_init_state)
    state_series_word2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=other_init_states[0])
    state_series_pos1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=other_init_states[0])
    state_series_pos2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=other_init_states[1])
    state_series_dep1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=other_init_states[1])
    state_series_dep2 = tf.reduce_max(state_series, axis=1)

state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

state_series = tf.concat([state_series1, state_series2], 1)

In [8]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

In [9]:
with tf.name_scope("dropout"):
    h_drop = tf.nn.dropout(y_hidden_layer, 0.3)

In [10]:
with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer, W) + b
    predictions = tf.argmax(logits, 1)

In [11]:
tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('biases')==-1):
            tv_regu.append(t)

with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

global_step = tf.Variable(0, name="global_step")

optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

In [12]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

In [13]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()

In [14]:
pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

In [15]:

rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [16]:
f = open('data/train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open('data/train_relations.txt'):
    relations.append(line.strip().split()[1])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [18]:
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [19]:
# model = tf.train.latest_checkpoint(model_dir)
# saver.restore(sess, model)

In [20]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd/word_embd


In [21]:
num_epochs = 20
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        if step%10==0:
            print("Step:", step, "loss:",loss)
        if step % 1000 == 0:
            saver.save(sess, model_dir + '/model')
            print("Saved Model")

Step: 10 loss: 2.87203
Step: 20 loss: 2.94049
Step: 30 loss: 2.79787
Step: 40 loss: 2.79359
Step: 50 loss: 2.92462
Step: 60 loss: 2.69989
Step: 70 loss: 2.50553
Step: 80 loss: 2.65037
Step: 90 loss: 2.57664
Step: 100 loss: 2.36885
Step: 110 loss: 2.40983
Step: 120 loss: 2.48324
Step: 130 loss: 1.99289
Step: 140 loss: 1.73668
Step: 150 loss: 2.39096
Step: 160 loss: 2.46612
Step: 170 loss: 2.05314
Step: 180 loss: 2.33783
Step: 190 loss: 2.19994
Step: 200 loss: 2.59979
Step: 210 loss: 2.01877
Step: 220 loss: 1.636
Step: 230 loss: 2.16837
Step: 240 loss: 1.73731
Step: 250 loss: 1.66309
Step: 260 loss: 1.93474
Step: 270 loss: 1.65532
Step: 280 loss: 1.3331
Step: 290 loss: 1.85862
Step: 300 loss: 1.37562
Step: 310 loss: 1.83715
Step: 320 loss: 1.48518
Step: 330 loss: 1.67063
Step: 340 loss: 2.25372
Step: 350 loss: 2.7181
Step: 360 loss: 1.91852
Step: 370 loss: 1.25768
Step: 380 loss: 1.55177
Step: 390 loss: 1.68344
Step: 400 loss: 1.32262
Step: 410 loss: 1.94006
Step: 420 loss: 1.34089
Step:

Step: 3240 loss: 0.578042
Step: 3250 loss: 0.217345
Step: 3260 loss: 0.114212
Step: 3270 loss: 0.255328
Step: 3280 loss: 0.205612
Step: 3290 loss: 0.126631
Step: 3300 loss: 0.150584
Step: 3310 loss: 0.187266
Step: 3320 loss: 0.297325
Step: 3330 loss: 0.149014
Step: 3340 loss: 0.101138
Step: 3350 loss: 0.12946
Step: 3360 loss: 0.788479
Step: 3370 loss: 0.135797
Step: 3380 loss: 0.252258
Step: 3390 loss: 0.270987
Step: 3400 loss: 0.110068
Step: 3410 loss: 0.100875
Step: 3420 loss: 0.100394
Step: 3430 loss: 0.197775
Step: 3440 loss: 0.37373
Step: 3450 loss: 0.135395
Step: 3460 loss: 0.11437
Step: 3470 loss: 0.13198
Step: 3480 loss: 0.206246
Step: 3490 loss: 0.157283
Step: 3500 loss: 0.128176
Step: 3510 loss: 0.130112
Step: 3520 loss: 0.108631
Step: 3530 loss: 0.124168
Step: 3540 loss: 0.395311
Step: 3550 loss: 0.259584
Step: 3560 loss: 0.10725
Step: 3570 loss: 0.130895
Step: 3580 loss: 0.100903
Step: 3590 loss: 0.302576
Step: 3600 loss: 0.176222
Step: 3610 loss: 0.125035
Step: 3620 loss: 

Step: 6330 loss: 0.228381
Step: 6340 loss: 0.0802837
Step: 6350 loss: 0.0718766
Step: 6360 loss: 0.0698685
Step: 6370 loss: 0.0836705
Step: 6380 loss: 0.0787375
Step: 6390 loss: 0.0777286
Step: 6400 loss: 0.0727035
Step: 6410 loss: 0.0709271
Step: 6420 loss: 0.0766888
Step: 6430 loss: 0.0755974
Step: 6440 loss: 0.0718128
Step: 6450 loss: 0.0704479
Step: 6460 loss: 0.0686913
Step: 6470 loss: 0.0669666
Step: 6480 loss: 0.0683517
Step: 6490 loss: 0.0656099
Step: 6500 loss: 0.0742228
Step: 6510 loss: 0.0670818
Step: 6520 loss: 0.0985332
Step: 6530 loss: 0.0702038
Step: 6540 loss: 0.0644812
Step: 6550 loss: 0.069682
Step: 6560 loss: 0.10884
Step: 6570 loss: 0.0655684
Step: 6580 loss: 0.194564
Step: 6590 loss: 0.0705782
Step: 6600 loss: 0.0670806
Step: 6610 loss: 0.063809
Step: 6620 loss: 0.0648018
Step: 6630 loss: 0.0716937
Step: 6640 loss: 0.0778472
Step: 6650 loss: 0.0688316
Step: 6660 loss: 0.0706609
Step: 6670 loss: 0.0758656
Step: 6680 loss: 0.083786
Step: 6690 loss: 0.0687488
Step: 67

Step: 9390 loss: 0.0726333
Step: 9400 loss: 0.0635252
Step: 9410 loss: 0.0634555
Step: 9420 loss: 0.0634227
Step: 9430 loss: 0.063042
Step: 9440 loss: 0.0877039
Step: 9450 loss: 0.0628113
Step: 9460 loss: 0.0624544
Step: 9470 loss: 0.0627328
Step: 9480 loss: 0.0629401
Step: 9490 loss: 0.0637829
Step: 9500 loss: 0.0644716
Step: 9510 loss: 0.0658092
Step: 9520 loss: 0.062915
Step: 9530 loss: 0.0761087
Step: 9540 loss: 0.0629772
Step: 9550 loss: 0.0628617
Step: 9560 loss: 0.0624458
Step: 9570 loss: 0.0635827
Step: 9580 loss: 0.0664681
Step: 9590 loss: 0.0670332
Step: 9600 loss: 0.0680933
Step: 9610 loss: 0.0626622
Step: 9620 loss: 0.0798335
Step: 9630 loss: 0.0634341
Step: 9640 loss: 0.0708747
Step: 9650 loss: 0.0612786
Step: 9660 loss: 0.0625679
Step: 9670 loss: 0.0610204
Step: 9680 loss: 0.0609803
Step: 9690 loss: 0.0602213
Step: 9700 loss: 0.0648566
Step: 9710 loss: 0.0608875
Step: 9720 loss: 0.0620151
Step: 9730 loss: 0.0692284
Step: 9740 loss: 0.0597157
Step: 9750 loss: 0.0641531
Ste

Step: 12340 loss: 0.0449373
Step: 12350 loss: 0.0406301
Step: 12360 loss: 0.0399375
Step: 12370 loss: 0.0381377
Step: 12380 loss: 0.037408
Step: 12390 loss: 0.0556019
Step: 12400 loss: 0.0440799
Step: 12410 loss: 0.0379918
Step: 12420 loss: 0.0387271
Step: 12430 loss: 0.0382483
Step: 12440 loss: 0.0374323
Step: 12450 loss: 0.0374527
Step: 12460 loss: 0.0379388
Step: 12470 loss: 0.243013
Step: 12480 loss: 0.0385961
Step: 12490 loss: 0.0408791
Step: 12500 loss: 0.0386611
Step: 12510 loss: 0.0396702
Step: 12520 loss: 0.0374399
Step: 12530 loss: 0.037948
Step: 12540 loss: 0.039682
Step: 12550 loss: 0.0416581
Step: 12560 loss: 0.0399892
Step: 12570 loss: 0.0377993
Step: 12580 loss: 0.0472667
Step: 12590 loss: 0.0468467
Step: 12600 loss: 0.0385915
Step: 12610 loss: 0.0381843
Step: 12620 loss: 0.0436951
Step: 12630 loss: 0.0388464
Step: 12640 loss: 0.0520183
Step: 12650 loss: 0.037705
Step: 12660 loss: 0.036456
Step: 12670 loss: 0.0374101
Step: 12680 loss: 0.0443635
Step: 12690 loss: 0.037463

Step: 15280 loss: 0.042355
Step: 15290 loss: 0.0422148
Step: 15300 loss: 0.0499156
Step: 15310 loss: 0.0423368
Step: 15320 loss: 0.104294
Step: 15330 loss: 0.0432414
Step: 15340 loss: 0.0418278
Step: 15350 loss: 0.0422188
Step: 15360 loss: 0.0451274
Step: 15370 loss: 0.0419246
Step: 15380 loss: 0.143821
Step: 15390 loss: 0.0424784
Step: 15400 loss: 0.0428406
Step: 15410 loss: 0.0412053
Step: 15420 loss: 0.0411216
Step: 15430 loss: 0.0421234
Step: 15440 loss: 0.0443036
Step: 15450 loss: 0.0433544
Step: 15460 loss: 0.0419378
Step: 15470 loss: 0.0416222
Step: 15480 loss: 0.0420453
Step: 15490 loss: 0.0408432
Step: 15500 loss: 0.0408033
Step: 15510 loss: 0.0410112
Step: 15520 loss: 0.0411865
Step: 15530 loss: 0.0407078
Step: 15540 loss: 0.0411532
Step: 15550 loss: 0.0418497
Step: 15560 loss: 0.0405876
Step: 15570 loss: 0.0414735
Step: 15580 loss: 0.0401146
Step: 15590 loss: 0.0453463
Step: 15600 loss: 0.0416337
Step: 15610 loss: 0.0401773
Step: 15620 loss: 0.0407705
Step: 15630 loss: 0.041

In [22]:
# training accuracy
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

training accuracy 99.95


In [23]:
f = open('data/test_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open('data/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [24]:
# test 
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("test accuracy", accuracy)

test accuracy 65.5350553506
